#   <center> **Extraction of relevant informations from minute**s</center>













 In this notebook, the API of Llama 2 will be used to extract key information from minutes (PVs). We will provide a context (minutes) to the large language model and pose a question to be answered based on that context. there are two options for this process:

    1️⃣ Initially, we will make use of the Replicate API free quota.

    2️⃣ Alternatively, we can directly leverage the pre-trained model from meta. However, this approach requires significant computational resources .

In [1]:
pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [2]:
import os
import replicate
import pandas as pd
import re

In [106]:
Marches_publics_ = pd.read_csv('Marches_publics_.csv')

# Full_Responce = pd.read_csv('/content/Full_Responce.csv')

In [108]:
Marches_publics_.head(1)

,Procédure,Catégorie,DateDePublication,Référence,Objet,AcheteurPublic,LieuDexécution,link,EXTRAIT_DE_PV
0,AOO ...,Fournitures,27/01/2024,50/DGSN/DEB/DF/SA/2023,Objet : l’acquisition de matériel de dortoir a...,Acheteur public : DIRECTION GENERALE DE LA SUR...,IFRANE,https://www.marchespublics.gov.ma/?page=entrep...,ROVAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...


In [5]:

question_list=["A quel prix était le marché remporte ?","quelle sont les  concurrents ? ","Qui a remporte le marche ?","A quelle Date d'ouverture des plis ?"]

# supplementary prompt or context to guide the model's response.

system_promp = ['give just the price with one sentence','','give just the Name with of the Company','give just the Date if it exist in one sentence']


The generated response may require some processing, so we define some functions to extract the vital information from the full response.

In [6]:
# useful fuctions

# to extract the price from string

def Extract_price(Context):

  # Context : sentence

  # pattern = r"\d{1,3}(?:,\d{3})*(?:\.\d+)?"

  pattern = r"\d{1,3}(?:\.\d{3})*(?:,\d+)?"

  match = re.search(pattern, Context)

  if match:

      extracted_value = match.group()

      # print(extracted_value)
  else:

    extracted_value = ''

    print("No match found.")

  return extracted_value


# to return the list of competitors from output of model

def Extract_list_competitors(full_response):

  # full_response : model responce

  List_of_competitors = full_response.split('\n')

  if len(List_of_competitors) > 2:

    List_of_competitors = List_of_competitors[2:]

  else :

    List_of_competitors = []

  # some process

  return List_of_competitors

# to extract the Bid opening date


def Extract_data(Responce):

  date_pattern = r'(\d{2}/\d{2}/\d{4})'

  try:
        
      match = re.search(date_pattern, Responce)

      date_string = match.group(1)

      # print(date_string)

  except:

    date_string = ''

    print('NO Bid Date')

  return date_string

# to extract Who won the market

def Extract_winning_company(full_response,List_of_competitors):

  pass

  # full_response = full_response.split()

  # for Company in List_of_competitors :

  #   if Company in full_response:

  #     return Company

  #   else :

  #     return  ''


In [22]:

#Set the REPLICATE_API_TOKEN environment variable


os.environ["REPLICATE_API_TOKEN"] = "r8_a*************************************M"

api = replicate.Client(api_token=os.environ["REPLICATE_API_TOKEN"])

# Full_Responce = pd.DataFrame()

In [23]:
# import pandas as pd

# Full_Responce_1 = pd.DataFrame()

# Full_Responce_1.loc[0, 'PRIX'] = 'full_response'

# Full_Responce_1

# Full_Responce_1.to_csv('Full_Responce____.csv',index=False)

In [24]:
for i in range(41,52):

  print("========================",i,"========================")

  for j in range(len(question_list)):

    print('***********************',j,'************************')

    # prompt :  input context + the question

    prompt = Marches_publics['EXTRAIT_DE_PV'][i]+"\n the question is "+question_list[j]

    output = api.run("meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",

            input={"prompt":prompt,

                  "system_prompt": system_promp[j],

                "temperature":0.15,  "max_length":5,"max_tokens":5}  ) #"repetition_penalty":1

    full_response = ""

    for item in output:

        full_response += item


    if j == 0 :

      PRICE = Extract_price(full_response)

      Marches_publics.loc[i, 'PRIX'] = PRICE

      Full_Responce.loc[i, 'PRIX'] = full_response

      # pass

    elif j == 1 :

      Concurrents = Extract_list_competitors(full_response)

      Marches_publics.loc[i, 'List Des Concurrents'] = ','.join(Concurrents)

      # Marches_publics.at[0, 'Concurrents'] = ','.join(Concurrents)


      Full_Responce.loc[i, 'List Des Concurrents'] = full_response

      # pass

    elif j == 2 :

      Marches_publics.loc[i, 'Qui a remporte le marche'] = full_response

      Full_Responce.loc[i, 'Qui a remporte le marche'] = full_response

      # pass
    else :

      Date = Extract_data(full_response)

      Marches_publics.loc[i, 'Date Ouverture des plis'] = Date

      Full_Responce.loc[i, 'Date Ouverture des plis'] = full_response

      # pass

======================== 41 ========================
*********************** 0 ************************
*********************** 1 ************************
*********************** 2 ************************
*********************** 3 ************************
======================== 42 ========================
*********************** 0 ************************
*********************** 1 ************************
*********************** 2 ************************
*********************** 3 ************************
NO Bid Date
======================== 43 ========================
*********************** 0 ************************
*********************** 1 ************************
*********************** 2 ************************
*********************** 3 ************************
======================== 44 ========================
*********************** 0 ************************
*********************** 1 ************************
*********************** 2 ************************
***********

In [25]:
# Marches_publics['EXTRAIT_DE_PV'][1]+"\n the question is "+question_list[1]

Marches_publics.to_csv('Marches_publics_51.csv',index=False)

Full_Responce.to_csv('Full_Responce_51.csv',index=False)

In [26]:
Full_Responce

,PRIX,List Des Concurrents,Qui a remporte le marche,Date Ouverture des plis
0,"The price of the contract was 552.000,00 MAD ...",Based on the information provided in the docu...,Based on the information provided in the docu...,Sure! Here's the answer to your question:\n\n...
1,"The price of the market was 487,000 dirhams.",Based on the information provided in the docu...,Based on the information provided in the docu...,Sure! Here's the answer to your question:\n\n...
2,"The price of the market was 976.900,00 MAD (M...",Based on the information provided in the docu...,"Based on the information provided, the compan...",Sure! Here's the answer to your question:\n\n...
3,The price of the market was 11h00 12h00.,Based on the information provided in the tend...,"Based on the information provided, it appears...",The answer to your question is:\n\n08/12/2023
4,Le marché a été remporté à un prix de 9 932 7...,Based on the information provided in the tend...,LAMALIF,La date d'ouverture des plis est le 29/12/2023.
5,"Le montant de l'acte est de 3.022.036,20 DH (...",Based on the information provided in the docu...,"Based on the information provided, it appears...",Sure! Here's the answer to your question:\n\n...
6,Le marché a été remporté à un prix de 2 475 8...,Based on the information provided in the text...,MAGIDEUTZ a remporté le marché avec un montan...,Sure! The answer to your question is:\n\n15/1...
7,Le marché a été remporté à un prix de 436 300...,Based on the information provided in the docu...,Le winner of the contract is CONSEILS SARL.,La date d'ouverture des plis est le 26/01/2024.
8,"According to the document you provided, the p...",Based on the information provided in the extr...,Based on the information provided in the extr...,Sure! Here's the answer to your question:\n\n...
9,"The price of the contract was MAD 3,133,200.00.",Based on the information provided in the extr...,"Based on the information provided, the compan...",Sure! Here's the answer to your question:\n\n...


The data will be processed for the first 52 public markets where some data has been extracted with a free quota of the API. The data can be useful for analysis purposes and extracting insights, of course, if the relevant data has been extracted from thousands of files.

In [38]:
import pandas as pd 

Marches_publics = pd.read_csv("Marches_publics_51.csv")  

Full_Responce = pd.read_csv("Full_Responce_51.csv")


In [109]:
Marches_publics['Objet'] = Marches_publics['Objet'].str.replace('Objet : ', '')

Marches_publics['AcheteurPublic'] = Marches_publics['AcheteurPublic'].str.replace('Acheteur public :', '')

In [39]:
# Marches_publics[:52]

# list of cities : Place of execution of public markets

moroccan_cities = [
    "Agadir","Al Hoceima","Azilal","Assilah","Azemmour","Beni Mellal","Beni Ansar","Benslimane","Berkane","Berrechid","Casablanca",
    "Chefchaouen","Chichaoua""Dakhla","El Jadida","El Kelaa des Sraghna","Errachidia","Essaouira","EL-HAJEB","Fes",
    "Guelmim","Ifrane","INEZGANE","Kenitra","Khenifra","Khouribga","Ksar el-Kebir","Laayoune","Larache","Marrakech","Martil","Meknes",
    "Mohammedia","Nador","Nouaceur","Ouarzazate","Oujda","Rabat","REHAMNA","Safi","Salé","Settat","Sidi Kacem",
    "Sidi Slimane","Sidi Bennour","Tangier","Tan-Tan","Tantan","Tanan","Taounate","Taza","Témara","Tétouan",
    "Tiznit","Youssoufia"]

i=0

for cities in  Marches_publics['LieuDexécution'][:52]:
    
    list_cities = []
    
    for _Cities in moroccan_cities:
        
        if _Cities.lower() in cities.lower():
            
            list_cities.append(_Cities.upper())
            
            
    Marches_publics['LieuDexécution'][i] =  list_cities      
    
    i+=1      


In [41]:
# Marches_publics['LieuDexécution'][:52]

In [42]:
# advenced function for price extraction with 4 re  pattern. 

import re

pattern_1 = r"\d{1,3}(?:,\d{3})*(?:\.\d+)?"

pattern_2= r"\d{1,3}(?:\.\d{3})*(?:,\d+)?"

pattern_3 = r"\b\d{1,3}(?:(?:\s\d{3})*(?:,\d{2})?)\b"

pattern_4 = r"\b\d{1,3}(?:(?:\s\d{3})*(?:,\d{2})?)\b"


patterns = []; patterns.append(pattern_1) ; patterns.append(pattern_2) ; patterns.append(pattern_3) ; patterns.append(pattern_4) ;


def Extract_price(Context):
    
    prices_of_patterns = []

  # Context : sentence

    for pat in patterns : 
        
        match = re.search(pat, Context)

        if match:

            extracted_value = match.group()


#             print(extracted_value)

            prices_of_patterns.append([extracted_value,len(extracted_value)])

        else:

            extracted_value = ''

#             print("No match found.")
            
            prices_of_patterns.append([extracted_value,0])
         
        
    max_second_element = float('-inf')  # Initialize max to negative infinity
    
    max_second_element_first_element = None
            
    for pair in prices_of_patterns:
        

#         print(pair)
        
        if len(pair)==2:
            
            second_element = pair[1]
            
            if second_element > max_second_element:

                max_second_element = second_element

                max_second_element_first_element = pair[0]
                
    if len(max_second_element_first_element)< 4 :
        
        max_second_element_first_element = ''
    
                   
    return max_second_element_first_element



In [43]:
index = 0

for i in Full_Responce["PRIX"][:52]:
    
    Marches_publics['PRIX'][index] =  Extract_price(i)
    
    index += 1
    
    

In [45]:
# Marches_publics['PRIX'][:52]



In [51]:
# extract list of competitors companys name 

sp_car=["*",'  ', "Société" ,"1.","2.","3.","4.","5.","6.","7.","8.","9.","0.","Lot 1:", "Lot 2:","Lot",")","(" ]

index = 0

def remove_special_characters(input_list):
    
    output_list = []
    
    for comp_name in input_list:
        
        for sp_car_ in sp_car:

            comp_name = comp_name.replace(sp_car_, '')
            
        output_list.append(comp_name)
            
    return output_list

concs = []

for comp_list in Marches_publics["List Des Concurrents"][:52]:

    if type(comp_list) != float:


        comp_list = comp_list.split(',')

        if '' in comp_list:

            comp_list = comp_list[:comp_list.index('')]




        for comp in comp_list  :

            if "NEANT" in comp :

                comp_list = []

                break

        for comp in comp_list  :

            if "Maitre d'Ouvrage" in comp:

                comp_list = comp_list[:comp_list.index(str(comp))]

                break
                
        for comp in comp_list  :

            if "MAD" in comp:

                comp_list = comp_list[:comp_list.index(str(comp))]

                break
                
                
        for comp in comp_list  :

            if "TTC" in comp:

                comp_list = comp_list[:comp_list.index(str(comp))]

                break

        for comp in comp_list  :

            if "Moroccan Dirham" in comp:

                comp_list=comp_list[:k.index(str(comp))]

                break

                
        for comp in comp_list  :

            if len(comp)>44:

                comp_list=comp_list[:comp_list.index(str(comp))]

                break
        
        for comp in comp_list :

            try:
                float(comp)
                
                comp_list.remove(comp)

            except ValueError:
                
                   pass

        comp_list = remove_special_characters(comp_list)
     
    else : 
        
        comp_list = []
        
    Marches_publics['List Des Concurrents'][index] =  comp_list
    
    index +=1
    
    

In [53]:
# Marches_publics['List Des Concurrents'][:52]

In [64]:
# extrait Qui a remporte le marche a partire de List Des Concurrents

index = 0

for list_compet in Marches_publics['List Des Concurrents'][:52]:
    
    
    if len(list_compet) > 0:
        
        for j in range(len(list_compet)) : 
            
#              i[j]
                
#              i[j].strip()
            
#              i[j].strip().replace('"', '') 
            
            if list_compet[j] in Marches_publics["Qui a remporte le marche"][index]:
            
                Marches_publics["Qui a remporte le marche"][index] = list_compet[j]
                
                break 
                
            elif list_compet[j].strip() in Marches_publics["Qui a remporte le marche"][index]:
                
                Marches_publics["Qui a remporte le marche"][index] = list_compet[j]
                
                break
                
            elif list_compet[j].strip().replace('"', '') in Marches_publics["Qui a remporte le marche"][index]:
                
                Marches_publics["Qui a remporte le marche"][index] = list_compet[j]
            
                break
                
            elif j==len(list_compet)-1 and list_compet[j] not in Marches_publics["Qui a remporte le marche"][index] : 
                
                Marches_publics["Qui a remporte le marche"][index] = ''
                
    else : 

        Marches_publics["Qui a remporte le marche"][index] = ''

                
    index+=1


In [66]:
#  Marches_publics["Qui a remporte le marche"][:52]

In [97]:
# advenced functions for bit date extraction with for pattern. 

from datetime import datetime

month_dict =  {
    'janvier': '01', 'février': '02', 'mars': '03', 'avril': '04', 'mai': '05', 'juin': '06',
    'juillet': '07', 'août': '08', 'septembre': '09', 'octobre': '10', 'novembre': '11', 'décembre': '12','décember':'12',
    'january': '01', 'february': '02', 'march': '03', 'april': '04', 'may': '05', 'june': '06',
    'july': '07', 'august': '08', 'september': '09', 'october': '10', 'november': '11', 'december': '12'
}

date_pattern = r'(?:january|february|march|april|may|june|july|august|september|october|november|december|janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre|décember)\s\d{1,2},\s\d{4}'
 

def format_date(date_str):
    
    date_parts = date_str.split(' ')  # Split the date string into parts
    
    month = month_dict.get(date_parts[0].lower(), '')  # Get numerical representation of the month
    
    day = date_parts[1][:-1]  # Extract the day part
    
    year = date_parts[2]  # Extract the year part

    return f'{day}/{month}/{year[-2:]}'  # Format the date as dd/mm/yy


def extract_date(sentence):

    
    day_month_pattern = '|'.join(month_dict.keys())
    
    pattern = rf'(\d{{1,2}})\s({day_month_pattern})\w*\s(\d{{4}})'
       
    match = re.search(pattern, sentence.lower())
    
    if match:
        
        day = match.group(1)
        
        month_name = match.group(2)

        year = match.group(3)
        
        month_num = month_dict.get(month_name)
        
        if month_num:
            
            date_str = f"{day} {month_num} {year}"
            
            date_obj = datetime.strptime(date_str, '%d %m %Y')
            
            return date_obj.strftime('%d/%m/%y')
        
    return None


def extract_date_from_sentence_2(sentence):
    

    match = re.search(date_pattern, sentence)
    
    if match:
 
        return format_date(match.group())

    return None

index = 0

for sentence in Full_Responce["Date Ouverture des plis"][:52]:

# for sentence in sentences:
    
    extracted_date = extract_date(sentence)
    
    if extracted_date:

        Marches_publics['Date Ouverture des plis'][index] = extracted_date
    
    elif extract_date_from_sentence_2(sentence.lower()): 

        
        Marches_publics['Date Ouverture des plis'][index] = extract_date_from_sentence_2(sentence.lower())
        
    else : 
        
        Marches_publics['Date Ouverture des plis'][index] = ''
        
    index += 1



In [99]:
# Marches_publics['Date Ouverture des plis'][:52]

In [110]:
Marches_publics[:52]

,Procédure,Catégorie,DateDePublication,Référence,Objet,AcheteurPublic,LieuDexécution,link,EXTRAIT_DE_PV,PRIX,List Des Concurrents,Qui a remporte le marche,Date Ouverture des plis
0,AOO ...,Fournitures,27/01/2024,50/DGSN/DEB/DF/SA/2023,l’acquisition de matériel de dortoir au profit...,DIRECTION GENERALE DE LA SURETE NATIONALE,[IFRANE],https://www.marchespublics.gov.ma/?page=entrep...,ROVAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...,"552.000,00","[ NISAHI EQUIPEMENTS 3E, VELSATIS TRAV, R...",ETS AIT BENNANI,12/12/23
1,AOO ...,Fournitures,27/01/2024,51/DGSN/DEB/DF/SA/2023,l’acquisition de matériel de couchage au profi...,DIRECTION GENERALE DE LA SURETE NATIONALE,[RABAT],https://www.marchespublics.gov.ma/?page=entrep...,ROVAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...,"487,000","[ RICHBOND, WE CONCEPT MAROC, COCHED, L...","""GETRADEQ""",14/12/23
2,AOO ...,Fournitures,27/01/2024,68/DGSN/DEB/DF/SA/2023,la fourniture de produits alimentaires pour us...,DIRECTION GENERALE DE LA SURETE NATIONALE,[ASSILAH],https://www.marchespublics.gov.ma/?page=entrep...,ROYAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...,"976.900,00","[ LFARIALI AYOUB, SOCIETE BATI MEZANINE SARL]",,12/01/24
3,AOO ...,Fournitures,26/01/2024,106/2023/S2A,FOURNITURE ET INSTALLATION DU MATERIELS ET EQU...,Société de Développement Local Agadir Souss M...,"[AGADIR, TANAN]",https://www.marchespublics.gov.ma/?page=entrep...,Royaume du Maroc\n\\noQoNEO O80 CoOOc oLElof[C...,,[ Beemer Concurrent ECG IEEE LICE],,
4,AOO ...,Travaux,26/01/2024,118/2023/S2A,"TRAVAUX DE REALISATION DU PARC URBAIN ""AL INBI...",Société de Développement Local Agadir Souss M...,"[AGADIR, TANAN]",https://www.marchespublics.gov.ma/?page=entrep...,Royaume du Maroc\n\\noQoNEO O80 CoOOc oLElof[C...,"9 932 700,00",[ SOCIETE LAMALIF with an offer of 9],,
5,AOO ...,Travaux,26/01/2024,39/2023,TRAVAUX DE CONSTRUCTION DU CENTRE D’INNOVATION...,AGENCE REGIONALE D'EXECUTION DES PROJETS CASA...,[CASABLANCA],https://www.marchespublics.gov.ma/?page=entrep...,KeptpBhl ae col\nRoyaume du Maroc\nAGENCE REGI...,"3.022.036,20",[],,
6,AOO ...,Fournitures,26/01/2024,DT4121313,Acquisition de quatre groupes électrogènes pou...,OFFICE NATIONALE DE L'ELECTRICITE ET DE L'EAU...,[MARRAKECH],https://www.marchespublics.gov.ma/?page=entrep...,ROYAUME DU MAROC Ay AL) ALL\nOffice National d...,"2 475 802,02","[ BRICO FIRST SERVICES, MAGIDEUTZ, SEHI, SOFA]",MAGIDEUTZ,
7,AOO ...,Services,26/01/2024,111/AREPFM/2023,ETUDE D’ALIMENTATION EN EAU POTABLE DES DOUARS...,AGENCE REGIONALE D'EXECUTION DES PROJETS DE L...,"[EL-HAJEB, IFRANE]",https://www.marchespublics.gov.ma/?page=entrep...,"ADCN eujliiiotl A. a2i5J Gag gal UiSgl ,\nFY a...","436 300,00","[ SARL CONSEILS, SAS LGHZAL, SCP ROCHDI LAHO...",,
8,AOO ...,Services,26/01/2024,134/2023,Assistance technique pour le suivi des projets...,REGIE AUTONOME MULTI - SERVICES D'AGADIR,"[AGADIR, TANAN]",https://www.marchespublics.gov.ma/?page=entrep...,RAMSA-EN-PS1 1-13\nRoyaume du Maroc\nRégie Aut...,"350 400,00",[],,
9,AOO ...,Fournitures,26/01/2024,133/2023,Normalisation des stations de traitement et de...,REGIE AUTONOME MULTI - SERVICES D'AGADIR,"[AGADIR, TANAN]",https://www.marchespublics.gov.ma/?page=entrep...,RAMSA-EN-PS11-13\nRoyaume du Maroc\nRégie Auto...,"3,133,200.00","[ COTRATEC, ART BINAV]",COTRATEC,


## Openai API -GPT-3

Here is the data retrieved using an open AI API for the same process of extracting information from minutes. In reality, it show some of performance than replicate api.

In [129]:
df=pd.read_json('df.json')


In [125]:
df.columns

Index(['quelle l'objet de l'offre ?', 'Qui a remporte le marche ?',
       'A quel prix était le marché remporte ?',
       'quelle sont les  concurrents ? ', 'qui est le maître d'ouvrage ?',
       'A quelle Date d'ouverture des plis ?'],
      dtype='object')

In [126]:
df

,quelle l'objet de l'offre ?,Qui a remporte le marche ?,A quel prix était le marché remporte ?,quelle sont les concurrents ?,qui est le maître d'ouvrage ?,A quelle Date d'ouverture des plis ?
0,L'offre a pour objet les études techniques et ...,Le concurrent retenu est Sté CABINET D'ETUDES ...,1680000.0,"[1- Sté BATIMENTTECH, 2- Sté GECIMED sarl, 3- ...",la Direction Provinciale de l'AREF Fés-Meknés ...,25-04-2022
1,L'appel d'offres a pour objet le gardiennage e...,SOCIETE M2C MAROC,,"[1- Sté BATIMENTTECH, 2- Sté GECIMED sarl, 3- ...",Le Centre National de Radioprotection est le m...,21-06-2022
2,L'offre porte sur l'achat de matériel d'enseig...,La STE TECH DATA GROUPE SARL (TDG),134474880.0,"[1- Sté BATIMENTTECH, 2- Sté GECIMED sarl, 3- ...",L'Académie Régionale d'Education et de Formati...,17-05-2022
3,Travaux de restauration de la mosquée Chleuh e...,1. Aucun offre admis (Manque des attestations ...,,"[1- Sté BATIMENTTECH , 2- Sté GECIMED sarl , 3...",Le maître d'ouvrage est la société Casablanca ...,23/06/2022
4,L'offre est pour l'achat du matériel d'enseign...,La STE DAKHCHI SARL,195069600.0,"[1- Sté BATIMENTTECH, 2- Sté GECIMED sarl, 3- ...",L'Académie Régionale d'Education et de Formati...,18-05-2022
5,l'offre porte sur l'installation d'une clôture...,"Ste Poly plante sarl, casablanca",,"[1- Sté BATIMENTTECH, 2- Sté GECIMED sarl, 3- ...",- Le maître d'ouvrage est le Ministère de l'Ag...,23/06/2022
6,La STE NAVARA BURO propose de fournir le matér...,La STE NAVARA BURO a remporté le marché pour u...,85681920.0,"[1- Sté BATIMENTTECH, 2- Sté GECIMED sarl, 3- ...",L'Académie Régionale d'Education et de Formati...,17-05-2022
7,L'objet de l'offre est l'achat de fournitures ...,La STE MEKNES IMPRESSION ET DISTRIBUTION SARL ...,19233840.0,"[1- Sté BATIMENTTECH, 2- Sté GECIMED sarl, 3- ...",L'Académie Régionale d'Education et de Formati...,23/06/2022
8,Direction des Services Techniques,Le marché a été remporté par la société XYZ.,,"[1. Sté BATIMENTTECH, 2. Sté GECIMED sarl, 3. ...",ADII,23-06-2022
9,La prestation objet de l’offre est la scanneri...,SASA BLD,221472000.0,"[1- Sté BATIMENTTECH, 2- Sté GECIMED sarl, 3- ...",OMPIC,23/06/2022


In [122]:
# nbr_concurents

## In summary

The approach yields excellent results in extracting relevant information from public documents using the Replicate API of a large language model employed as a question-answering model for public market analysis.

Possible analyses on this type of data with a sufficient quantity could include:


    Analyzing the presence of a company or companies among competitors in different markets.
    
    Analyzing the price at which the market is won based on the Public Category/Place of Execution.
    
    
    For a future procurement need, identifying the prices proposed by different companies to win a particular type of contract (like catering services), considering factors such as time and geographical location. This analysis aims to determine if the proposed prices by companies are increasing steadily or fluctuating.


    Price competitiveness analysis: Comparing prices offered by competitors, identifying price trends, detecting any market distortions.

    Competitor performance analysis: Evaluating the strengths and weaknesses of competitors, identifying market leaders, spotting winning strategies.

    Geographic analysis: Studying the locations of market execution sites, identifying high-potential areas, adapting strategies based on regions.


# LLama 2 pre-trained model 

In [1]:
# Download the model to the repository

# ! wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q3_K_S.bin?download=true
! wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin?download=true

--2024-02-11 15:08:42--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin?download=true
Resolving huggingface.co (huggingface.co)... 13.32.208.45, 13.32.208.111, 13.32.208.55, ...
Connecting to huggingface.co (huggingface.co)|13.32.208.45|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/3bfdde943555c78294626a6ccd40184162d066d39774bd2c98dae24943d32cc3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q8_0.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1707923322&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNzkyMzMyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQx

In [2]:
# pip install llama-cpp-python

!pip install llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=296593 sha256=110ae745bbe5e54ad0179ff51a20c0e577fbe36b302cb28dce2d01be611b4485
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [3]:
from llama_cpp import Llama

In [4]:
# define n_ctx manually to permit larger contexts

# Setting up a Llama language model.

LLM = Llama(model_path="llama-2-7b-chat.ggmlv3.q8_0.bin?download=true",n_ctx=2048)


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [ ]:
output = LLM(prompt)


In [14]:
# display the response

output["choices"][0]["text"]

"\nPlease refer to the table above for the answer. According to the information provided, Sté CABINET D'ETUDES EL YOUSSEF! sarl has been retained as the successful bidder."